In [1]:
import os
os.environ['PROJ_LIB'] = r'C:\Users\osori050\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Library\share\proj'

In [153]:
import io
import zipfile
import requests
import geopandas as gpd
import pandas as pd
from itertools import product
import numpy as np
import math
import shutil
from scipy.stats import zscore
import fiona
from fiona.crs import from_epsg
from shapely import geometry
import psycopg2

In [159]:
# Set workspace
os.chdir(r'E:\ArcGIS_2\Lab4')
wksp = os.getcwd()

In [4]:
# Retrieve MN territories Data
territories_output = requests.post(r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/bdry_mn_city_township_unorg/shp_bdry_mn_city_township_unorg.zip')
zipfile.ZipFile(io.BytesIO(territories_output.content)).extractall(wksp)

In [5]:
# Extract only the cities
arcpy.analysis.Select(
    in_features="city_township_unorg.shp",
    out_feature_class="cities.shp",
    where_clause="CTU_CLASS = 'CITY'"
)

<Result 'E:\\ArcGIS_2\\Lab4\\cities.shp'>

In [6]:
# Select only places with stink bug presence
arcpy.analysis.Select(
    in_features="sbug_points.shp",
    out_feature_class="sbug_presence.shp",
    where_clause="Adults <> 0 Or Nymphs <> 0"
)

<Result 'E:\\ArcGIS_2\\Lab4\\sbug_presence.shp'>

In [7]:
# Select the cities with stink bug presence
selection = arcpy.SelectLayerByLocation_management('city_township_unorg.shp', "INTERSECT", 'sbug_presence.shp', 0, "NEW_SELECTION")
subselection = arcpy.SelectLayerByAttribute_management(selection, "SUBSET_SELECTION", "CTU_CLASS = 'CITY'")
arcpy.CopyFeatures_management(selection, 'sbug_cities.shp')

<Result 'E:\\ArcGIS_2\\Lab4\\sbug_cities.shp'>

In [18]:
# Cities with stink bug presence data frame
affected_cities = gpd.read_file('sbug_cities.shp')
affected_cities.drop(affected_cities.iloc[:, np.r_[0, 2:10]], axis=1, inplace=True) 
affected_cities['Presence'] = 1

In [19]:
# All cities data frame
cities_gdf = gpd.read_file('cities.shp')
cities_gdf.drop(cities_gdf.iloc[:, np.r_[0, 2:6, 7:9]], axis=1, inplace=True) 

# Add presence to cities data frame
affected_cities['FEATURE_NA'] = affected_cities['FEATURE_NA'].astype(str)
cities_gdf = cities_gdf.join(affected_cities.set_index('FEATURE_NA'), on='FEATURE_NA', how='left')
cities_gdf['Presence'] = cities_gdf['Presence'].fillna(0)

In [20]:
# Calculate z-values for population
col = ['POPULATION']
z_val = cities_gdf[col].apply(zscore)
cities_gdf = cities_gdf.join(z_val, rsuffix=" (Z)")

# Move the distribution so the starting poing is zero
cities_gdf["POPULATION (Z)"] = cities_gdf["POPULATION (Z)"].apply(lambda x: abs(cities_gdf["POPULATION (Z)"].min()) + x)

# Only cities with presencce
cities_with_presence = cities_gdf[cities_gdf["Presence"] == 1].copy()

In [75]:
# Combine each city with the others in MN except with itself
combinations = product(cities_with_presence.values, cities_gdf.values)
df_combined = pd.DataFrame([*combinations], columns=['From', 'To'])
df_combined[['From', 'To']] = df_combined[['From', 'To']].applymap(lambda x: x[0])
df_combined = df_combined.loc[df_combined['From'] != df_combined['To']]
df_combined.reset_index(drop=True, inplace=True)

In [76]:
# Add geometry to 'From' and 'To' cities
df_combined = df_combined.merge(cities_gdf, left_on='From', right_on='FEATURE_NA', how='inner')
df_combined = df_combined.merge(cities_gdf, left_on='To', right_on='FEATURE_NA', how='left')
df_combined.drop(df_combined.iloc[:, np.r_[2, 7]], axis=1, inplace=True)
df_combined.drop_duplicates(inplace=True) # Remove duplicates generated in the previous join
df_combined.reset_index(drop=True, inplace=True)

In [77]:
# Calculate centroids
df_combined['centroid_from'] = ''
df_combined['centroid_to'] = ''
for i in range(len(df_combined)):
    df_combined.at[i, 'centroid_from'] = df_combined['geometry_x'][i].centroid.coords[0]
    df_combined.at[i, 'centroid_to'] = df_combined['geometry_y'][i].centroid.coords[0]
    
# Calculate euclidean distance
df_combined['Distance'] = ''
for i in range(len(df_combined)):
    df_combined.at[i, 'Distance'] = math.dist(df_combined.at[i, 'centroid_from'], df_combined.at[i, 'centroid_to']) / 1000 # km

In [78]:
df_combined.drop(df_combined.iloc[:, np.r_[3, 7, 10:12]], axis=1, inplace=True)

In [106]:
# Create copy to avoid running the latter cell, which takes long
iteration_df = df_combined.copy()
iteration_df.drop(iteration_df.iloc[:, np.r_[2, 5]], axis=1, inplace=True)

# Rename columns
iteration_df = iteration_df.rename(columns={'Presence_x': 'Presence From', 
                                            'POPULATION (Z)_x': 'W From', 
                                            'Presence_y': 'Presence To', 
                                            'POPULATION (Z)_y': 'W To'})

In [107]:
## Calculate probability of transition
iteration_df['Force'] = (iteration_df['W From']*iteration_df['W To'])/iteration_df['Distance']

# Calculate total force for each from-city
force_per_from_city = iteration_df.groupby('From').agg({'Force': 'sum'}).reset_index()
force_per_from_city = force_per_from_city.rename(columns={'Force': 'Total force per from-city'})
iteration_df = iteration_df.merge(force_per_from_city, on='From', how='left')

# Probability of transition
iteration_df['Prob_trans'] = 0
for i in range(len(iteration_df)):
    if iteration_df.at[i, 'Total force per from-city'] != 0:
        iteration_df.at[i, 'Prob_trans'] = iteration_df.at[i, 'Force']/iteration_df.at[i, 'Total force per from-city']
    else:
        iteration_df.at[i, 'Total force per from-city'] = 0

In [108]:
# Monte carlo simulation
# iteration_df['Prob_trans'] = affected_cities['FEATURE_NA'].astype(str)
for i in range (0,100):
    # Get draw from flat distribution
    transition_draw = np.random.random()
    
    # Predict affected cities
    for index, row in iteration_df.iterrows():
        if row["Prob_trans"] > transition_draw:
            iteration_df.loc[index, "Presence To"] = 1

In [91]:
# Calculate risk of invasion for each city
risk = iteration_df.groupby('To').agg({'Prob_trans': 'sum'}).reset_index()
risk = risk.rename(columns={'Prob_trans': 'Risk'})

In [116]:
# Extract potential cities affected into a new data frame
predicted = iteration_df[iteration_df["Presence To"] == 1].copy()
predicted = predicted.groupby('To').agg({'Presence To': 'min'}).reset_index()

In [139]:
# Read city shapefile again and join the prediction and risk
cities_gdf = gpd.read_file('cities.shp')
cities_gdf.drop(cities_gdf.iloc[:, np.r_[0, 2:9]], axis=1, inplace=True) 
cities_gdf = cities_gdf.rename(columns={'FEATURE_NA': 'Name'})
cities_gdf = cities_gdf.merge(predicted, left_on='Name', right_on='To', how='left')
cities_gdf = cities_gdf.merge(risk, left_on='Name', right_on='To', how='left')
cities_gdf.drop(cities_gdf.iloc[:, np.r_[2, 4]], axis=1, inplace=True) 
cities_gdf = cities_gdf.rename(columns={'Presence To': 'Presence'})
cities_gdf['Presence'] = cities_gdf['Presence'].fillna(0)

In [151]:
# Define the output shapefile schema
schema = {
    'geometry': 'Polygon',
    'properties': {
        'Name': 'str',
        'Presence': 'int',
        'Risk': 'float'
    }
}

# Set the CRS of the GeoDataFrame
crs = from_epsg(26915)

# Open the output shapefile and write the GeoDataFrame to it
with fiona.open('cities_results.shp', 'w', driver='ESRI Shapefile', schema=schema, crs=crs) as output:
    for index, row in cities_gdf.iterrows():
        output.write({
            'geometry': geometry.mapping(row.geometry),
            'properties': {
                'Name': row['Name'],
                'Presence': row['Presence'],
                'Risk': row['Risk']
            }
        })

In [160]:
# Project to WGS 1984
sr = arcpy.SpatialReference(4326)
arcpy.Project_management('cities_results', 'cities_projected.shp', sr)

<Result 'E:\\ArcGIS_2\\Lab4\\cities_projected.shp'>

In [170]:
# Connect to postgresql database
connection = psycopg2.connect(host = '34.27.219.64',
                              port = '5432',
                              database = 'lab1',
                              user = 'postgres',
                              password = 'student',
                             )

In [171]:
data = ("cities_projected.shp")
fields = ["Name", "Presence", "Risk", "Shape@WKT"]

# Create SQL table
cursor = connection.cursor()
cursor.execute("DROP TABLE IF EXISTS cities")
cursor.execute("""
    CREATE TABLE cities (
        id SERIAL,
        Name VARCHAR,
        Presence INTEGER,
        Risk DOUBLE PRECISION)
""")

cursor.execute("""
    SELECT AddGeometryColumn('cities', 'geom', 4326, 'MULTIPOLYGON', 2)
""")

# Populate PostGIS
with arcpy.da.SearchCursor(data, fields) as da_cursor:
    for row in da_cursor:
        wkt = row[3]
        cursor.execute("INSERT INTO cities (Name, Presence, Risk, geom) VALUES (%s, %s, %s, ST_GeomFromText(%s, 4326))", (row[0], row[1], row[2], wkt))

connection.commit()
connection.close()